In [ ]:
# Insert the library
import boto3
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import matplotlib.pyplot as plt

# Initialization boto3
s3_client = boto3.client('s3')

# Set the variable bucket and file
bucket_name = 'yourfirehosebucket'
bucket_models = 'yourmodelsbucket'
file_key = 'yourdatajson'

# Read file JSON from S3 Bucket
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
content = response['Body']
json_data = json.loads(content.read())

# Convert JSON to Data Frame
df = pd.DataFrame(json_data)
print(df.head())

# Convert timestamp coloumn to Datetime Format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Convert coloumn timestamp to integer format(number of seconds since epoch)
df['timestamp'] = df['timestamp'].astype('int64') // 10**9

# Define features all data and the target is fire_intensity
features = ['timestamp', 'temperature', 'humidity', 'fire_intensity', 'gas_concentration', 'wind_speed', 'distance']
target = 'fire_intensity'

# Delete coloumn target from features
X = df[features].drop(columns=[target])
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Prediction test data
y_pred = model.predict(X_test)

# Model Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

# Visualization prediction and actual result
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('Actual Fire Intensity')
plt.ylabel('Predicted Fire Intensity')
plt.title('Actual vs Predicted Fire Intensity')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)
plt.show()

# Save model to pkl local file

import pickle

# Simpan model ke file lokal
with open("model.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model berhasil disimpan ke model.pkl")


# Upload model to S3 Model

import boto3

# Nama bucket di S3
bucket_name = "lks-bucketmodell-jatengg"
model_filename = "model.pkl"
s3_key = f"models/{model_filename}"  # Lokasi di S3

# Buat koneksi ke S3
s3 = boto3.client("s3")

# Upload file model ke S3
s3.upload_file(model_filename, bucket_name, s3_key)

print(f"Model berhasil diunggah ke s3://{bucket_name}/{s3_key}")

